# IMPORTS

In [7]:
from tifffile import imread

"""Core functionality for CellProfiler feature extraction using cp_measure package."""

from itertools import combinations #used to generate pairs of channels for colocalization

import numpy as np
import pandas as pd
from cp_measure.bulk import (
    get_core_measurements, 
    get_correlation_measurements,
    get_multimask_measurements
)
 #contains core measurements like, intensity, shape, texture etc.
from cp_measure.core import (
    measurecolocalization,
    # measureobjectneighbors,
    # measureobjectoverlap
) #contains colocalization, neighbor, overlap measurements -- more specialized than the bulk ones.
from cp_measure.multimask import (
    measureobjectneighbors,
    measureobjectoverlap
) #contains colocalization, neighbor, overlap measurements -- more specialized than the bulk ones.

#  VISUALIZING DATA

### Example image (From cp measure README)

In [ ]:
size = 200
rng = np.random.default_rng(42)
pixels = rng.integers(low=0, high=10, size=(size, size))

masks = np.zeros_like(pixels)
masks[5:-6, 5:-6] = 1
masks
pixels

In [ ]:
#altered
masks = np.zeros_like(pixels)
masks[5:-6, 5:-6] = 1
masks[4:-10, 4:-10] = 2
masks[3:-8, 3:-8] = 3
masks

### Example analysis data

In [8]:
data_phenotype = imread('/lab/barcheese01/amallar/brieflow/example_analysis/analysis_root/phenotype_process/images/W-A1_T-1__illumination_corrected.tiff')
nuclei = imread('/lab/barcheese01/amallar/brieflow/example_analysis/analysis_root/phenotype_process/images/W-A1_T-1__nuclei.tiff')
cells = imread('/lab/barcheese01/amallar/brieflow/example_analysis/analysis_root/phenotype_process/images/W-A1_T-1__cells.tiff')
cytoplasms = imread('/lab/barcheese01/amallar/brieflow/example_analysis/analysis_root/phenotype_process/images/W-A1_T-1__identified_cytoplasms.tiff')

#### *Specs*:

In [6]:
print(f"Phenotype shape: {data_phenotype.shape}")
print(f"\nPhenotype CHANNEL 1 specs")
print(f"max:{data_phenotype[0].max()}")
print(f"min:{data_phenotype[0].min()}")
print(f"mean:{data_phenotype[0].mean()}")
print(f"var:{data_phenotype[0].var()}")
print(f"std:{data_phenotype[0].std()}")
print(f"\nPhenotype CHANNEL 2 specs")
print(f"max:{data_phenotype[1].max()}")
print(f"min:{data_phenotype[1].min()}")
print(f"mean:{data_phenotype[1].mean()}")
print(f"var:{data_phenotype[1].var()}")
print(f"std:{data_phenotype[1].std()}")
print(f"\nPhenotype CHANNEL 3 specs")
print(f"max:{data_phenotype[2].max()}")
print(f"min:{data_phenotype[2].min()}")
print(f"mean:{data_phenotype[2].mean()}")
print(f"var:{data_phenotype[2].var()}")
print(f"std:{data_phenotype[2].std()}")
print(f"\nPhenotype CHANNEL 4 specs")
print(f"max:{data_phenotype[3].max()}")
print(f"min:{data_phenotype[3].min()}")
print(f"mean:{data_phenotype[3].mean()}")
print(f"var:{data_phenotype[3].var()}")
print(f"std:{data_phenotype[3].std()}")

print(f"\n Nuclei Mask specs")
print(f"shape:{nuclei.shape}")
print(f"max:{nuclei.max()}")
print(f"min:{nuclei.min()}")
print(f"mean:{nuclei.mean()}")
print(f"var:{nuclei.var()}")
print(f"std:{nuclei.std()}")

print(f"\n Cells Mask specs")
print(f"shape:{cells.shape}")
print(f"max:{cells.max()}")
print(f"min:{cells.min()}")
print(f"mean:{cells.mean()}")
print(f"var:{cells.var()}")
print(f"std:{cells.std()}")

print(f"\n Cytoplasms Mask specs")
print(f"shape:{cytoplasms.shape}")
print(f"max:{cytoplasms.max()}")
print(f"min:{cytoplasms.min()}")
print(f"mean:{cytoplasms.mean()}")
print(f"var:{cytoplasms.var()}")
print(f"std:{cytoplasms.std()}")




Phenotype shape: (4, 2400, 2400)

Phenotype CHANNEL 1 specs
max:16944
min:23
mean:520.915921875
var:557514.7078319102
std:746.6690751811743

Phenotype CHANNEL 2 specs
max:3054
min:100
mean:268.0464524305556
var:12047.11487793559
std:109.75934984289762

Phenotype CHANNEL 3 specs
max:1447
min:76
mean:237.05985190972223
var:9914.600263755856
std:99.5720857658202

Phenotype CHANNEL 4 specs
max:19102
min:630
mean:2477.066137326389
var:1002526.760608668
std:1001.2625832461073

 Nuclei Mask specs
shape:(2400, 2400)
max:1088
min:0
mean:146.08085104166668
var:85907.5041054701
std:293.0998193542093

 Cells Mask specs
shape:(2400, 2400)
max:1088
min:0
mean:245.89161510416668
var:115083.79546712409
std:339.2400263340458

 Cytoplasms Mask specs
shape:(2400, 2400)
max:1088
min:0
mean:100.23690642361112
var:58603.373247742355
std:242.0813360169312


#### *Shape*:

In [ ]:
print("\nEntire phenotype image (4, 2400, 2400):")
print(data_phenotype)
print("\nFirst channel of phenotype image:")
print(data_phenotype[0,])
print("\nFirst row of first channel:")
print(data_phenotype[0, 0])
print("\nFirst pixel in first channel:")
print(data_phenotype[0, 0, 0])  

print("\nFull 2D nuclei mask")
print(nuclei)
print("\nFirst row of pixels in mask:")
print(nuclei[0])
print("\nFirst pixel in mask:")
print(nuclei[0, 0])

### Small example:

In [ ]:

import numpy as np

# tiny 3D array (2 channels, 3x3 each)
tiny_array = np.array([
    # First mask (index 0)
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],
    
    # Second mask (index 1)
    [[10, 11, 12],
     [13, 14, 15],
     [16, 17, 18]]
])

print("Array shape:", tiny_array.shape)  # Should be (2, 3, 3)
print("\nEntire array:")
print(tiny_array)
print("\nFirst channel (index 0):")
print(tiny_array[0])
print("\nSecond channel (index 1):")
print(tiny_array[1])
print("\nFirst row of first channel:")
print(tiny_array[0, 0])
print("\nSingle value (row 1, col 1 of first channel):")
print(tiny_array[0, 1, 1])  # Should be 5

# UNIT TESTING

In [14]:
measurements = get_core_measurements()
print(measurements.keys())
# dict_keys(['radial_distribution', 'radial_zernikes', 'intensity', 'sizeshape', 'zernike', 'ferret', 'texture', 'granularity'])

dict_keys(['radial_distribution', 'radial_zernikes', 'intensity', 'sizeshape', 'zernike', 'ferret', 'texture', 'granularity'])


### TEST: Mock image

In [13]:
measurements = get_core_measurements()

size = 200
rng = np.random.default_rng(42)
pixels = rng.integers(low=0, high=10, size=(size, size))

masks = np.zeros_like(pixels)
masks[5:-6, 5:-6] = 1
masks[4:-10, 4:-10] = 2
masks[3:-15, 3:-15] = 3

results_mock = {}
for name, v in measurements.items():
    print(f'{name}')
    results_mock = {**results_mock, **v(masks, pixels)}
    break

df = pd.DataFrame(results_mock)

df

radial_distribution


/lab/barcheese01/amallar/miniconda3/envs/cpmeasure_new/lib/python3.10/site-packages/cp_measure/core/measureobjectintensitydistribution.py:223: RuntimeWarning: invalid value encountered in divide
  fraction_at_distance = histogram / sum_by_object_per_bin
/lab/barcheese01/amallar/miniconda3/envs/cpmeasure_new/lib/python3.10/site-packages/cp_measure/core/measureobjectintensitydistribution.py:233: RuntimeWarning: invalid value encountered in divide
  fraction_at_bin = number_at_distance / sum_by_object_per_bin
/lab/barcheese01/amallar/miniconda3/envs/cpmeasure_new/lib/python3.10/site-packages/cp_measure/core/measureobjectintensitydistribution.py:279: RuntimeWarning: invalid value encountered in divide
  radial_means = numpy.ma.masked_array(radial_values / pixel_count, mask)


,RadialDistribution_FracAtD_1of4,RadialDistribution_MeanFrac_1of4,RadialDistribution_RadialCV_1of4,RadialDistribution_FracAtD_2of4,RadialDistribution_MeanFrac_2of4,RadialDistribution_RadialCV_2of4,RadialDistribution_FracAtD_3of4,RadialDistribution_MeanFrac_3of4,RadialDistribution_RadialCV_3of4,RadialDistribution_FracAtD_4of4,RadialDistribution_MeanFrac_4of4,RadialDistribution_RadialCV_4of4
0,0.000597,0.893534,0.0,0.007615,1.139256,0.374959,0.015529,0.860441,0.267419,0.976258,1.001702,0.05063
1,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.00000
2,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.00000


### TEST: Example Analysis Data

In [15]:
measurements = get_core_measurements()

data_phenotype = imread('/lab/barcheese01/amallar/brieflow/example_analysis/analysis_root/phenotype_process/images/W-A1_T-1__illumination_corrected.tiff')
nuclei = imread('/lab/barcheese01/amallar/brieflow/example_analysis/analysis_root/phenotype_process/images/W-A1_T-1__nuclei.tiff')

results = {}    
for name, v in measurements.items():
    print(f'{name}')
    results = {**results, **v(nuclei, data_phenotype[0])}
    break

df = pd.DataFrame(results)
df

radial_distribution


/lab/barcheese01/amallar/miniconda3/envs/cpmeasure_new/lib/python3.10/site-packages/cp_measure/core/measureobjectintensitydistribution.py:223: RuntimeWarning: invalid value encountered in divide
  fraction_at_distance = histogram / sum_by_object_per_bin
/lab/barcheese01/amallar/miniconda3/envs/cpmeasure_new/lib/python3.10/site-packages/cp_measure/core/measureobjectintensitydistribution.py:233: RuntimeWarning: invalid value encountered in divide
  fraction_at_bin = number_at_distance / sum_by_object_per_bin
/lab/barcheese01/amallar/miniconda3/envs/cpmeasure_new/lib/python3.10/site-packages/cp_measure/core/measureobjectintensitydistribution.py:279: RuntimeWarning: invalid value encountered in divide
  radial_means = numpy.ma.masked_array(radial_values / pixel_count, mask)


,RadialDistribution_FracAtD_1of4,RadialDistribution_MeanFrac_1of4,RadialDistribution_RadialCV_1of4,RadialDistribution_FracAtD_2of4,RadialDistribution_MeanFrac_2of4,RadialDistribution_RadialCV_2of4,RadialDistribution_FracAtD_3of4,RadialDistribution_MeanFrac_3of4,RadialDistribution_RadialCV_3of4,RadialDistribution_FracAtD_4of4,RadialDistribution_MeanFrac_4of4,RadialDistribution_RadialCV_4of4
0,0.141567,2.523002,0.030041,0.451404,2.520499,0.041869,0.225025,0.690467,0.036895,0.182004,0.414688,0.067156
1,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
2,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
3,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
4,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1083,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1084,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1085,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1086,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000


### MISC

In [ ]:
size2 = 2400
rng2 = np.random.default_rng(42)
mask_normalized = rng.integers(low=0, high=10, size=(size2, size2))

nuclei_normalized = np.zeros_like(mask_normalized)

for mask in nuclei:
    for index in mask:
        if mask[index] != 0:
            #normalize label value to 1
            nuclei_normalized[index] = 1

In [ ]:
nuclei_normalized 

In [ ]:
results_mock

In [ ]:
results

# FULL FEATURE EXTRACTION TEST

### Mock Data

In [9]:
import numpy as np
from scipy import stats

def generate_mock_channel(size, min_val, max_val, mean, std, random_state=None):
    """Generate mock channel data with specified statistics"""
    rng = np.random.RandomState(random_state)
    
    # Generate normal distribution then transform to match desired statistics
    data = rng.normal(loc=0, scale=1, size=size)
    data = stats.zscore(data)  # standardize
    data = data * std + mean  # scale and shift
    
    # Clip to min/max range
    data = np.clip(data, min_val, max_val)
    
    return data.astype(np.float32)

def generate_mock_mask(size, max_label, mean, std, random_state=None):
    """Generate mock mask with specified number of labels"""
    rng = np.random.RandomState(random_state)
    
    # Calculate approximate number of non-zero pixels needed
    total_pixels = size[0] * size[1]
    target_filled = int((mean * total_pixels) / max_label)
    
    # Create empty mask
    labeled = np.zeros(size, dtype=np.int32)
    
    # Generate random locations for each label
    for label in range(1, max_label + 1):
        # Create a small cluster for this label
        center_y = rng.randint(0, size[0])
        center_x = rng.randint(0, size[1])
        
        # Create a random cluster size
        cluster_size = max(1, int(target_filled / max_label))
        
        # Generate random offsets from center
        y_offsets = rng.normal(0, size[0]/10, cluster_size).astype(int)
        x_offsets = rng.normal(0, size[1]/10, cluster_size).astype(int)
        
        # Apply offsets and clip to image boundaries
        y_coords = np.clip(center_y + y_offsets, 0, size[0]-1)
        x_coords = np.clip(center_x + x_offsets, 0, size[1]-1)
        
        # Assign label to these positions (only where not already labeled)
        for y, x in zip(y_coords, x_coords):
            if labeled[y, x] == 0:  # only if position is empty
                labeled[y, x] = label
    
    return labeled

# Generate mock data
size = (200, 200)
mock_size = (4, *size)

# Generate phenotype channels
mock_phenotype = np.zeros(mock_size, dtype=np.float32)

# Channel specs
channel_specs = [
    {'min': 23, 'max': 16944, 'mean': 520.92, 'std': 746.67}, # channel 1
    {'min': 100, 'max': 3054, 'mean': 268.05, 'std': 109.76}, # channel 2
    {'min': 76, 'max': 1447, 'mean': 237.06, 'std': 99.57}, # channel 3
    {'min': 630, 'max': 19102, 'mean': 2477.07, 'std': 1001.26} # channel 4
]

# Generate each channel
for i, specs in enumerate(channel_specs):
    mock_phenotype[i] = generate_mock_channel(
        size=size,
        min_val=specs['min'],
        max_val=specs['max'],
        mean=specs['mean'],
        std=specs['std'],
        random_state=i
    )

# Generate masks
max_labels = 90  # scaled down from 1088 for 200x200
mask_specs = [
    {'name': 'nuclei', 'mean': 146.08, 'std': 293.10},
    {'name': 'cells', 'mean': 245.89, 'std': 339.24},
    {'name': 'cytoplasm', 'mean': 100.24, 'std': 242.08}
]

mock_masks = {}
for i, specs in enumerate(mask_specs):
    mock_masks[specs['name']] = generate_mock_mask(
        size=size,
        max_label=max_labels,
        mean=specs['mean'],
        std=specs['std'],
        random_state=i+10
    )

# Verify statistics
print("Mock Phenotype Data:")
for i in range(4):
    print(f"\nChannel {i+1}:")
    print(f"Shape: {mock_phenotype[i].shape}")
    print(f"Min: {mock_phenotype[i].min():.2f}")
    print(f"Max: {mock_phenotype[i].max():.2f}")
    print(f"Mean: {mock_phenotype[i].mean():.2f}")
    print(f"Std: {mock_phenotype[i].std():.2f}")

print("\nMock Mask Data:")
for name, mask in mock_masks.items():
    print(f"\n{name.capitalize()}:")
    print(f"Shape: {mask.shape}")
    print(f"Min: {mask.min()}")
    print(f"Max: {mask.max()}")
    print(f"Mean: {mask.mean():.2f}")
    print(f"Std: {mask.std():.2f}")

Mock Phenotype Data:

Channel 1:
Shape: (200, 200)
Min: 23.00
Max: 3376.77
Mean: 633.98
Std: 588.74

Channel 2:
Shape: (200, 200)
Min: 100.00
Max: 728.82
Mean: 270.97
Std: 104.00

Channel 3:
Shape: (200, 200)
Min: 76.00
Max: 651.46
Mean: 239.25
Std: 95.10

Channel 4:
Shape: (200, 200)
Min: 630.00
Max: 6680.49
Mean: 2489.07
Std: 974.56

Mock Mask Data:

Nuclei:
Shape: (200, 200)
Min: 0
Max: 90
Mean: 24.89
Std: 26.00

Cells:
Shape: (200, 200)
Min: 0
Max: 90
Mean: 26.49
Std: 24.73

Cytoplasm:
Shape: (200, 200)
Min: 0
Max: 90
Mean: 23.64
Std: 27.80


### Windows from real imgs

In [10]:
# Take a 200x200 window from the center of each image
start_y = 1100  # Center-ish of 2400x2400
start_x = 1100
window_size = 200

# Extract windows from phenotype data
w_phenotype = data_phenotype[:, start_y:start_y+window_size, start_x:start_x+window_size]

# Extract windows from masks
w_nuclei = nuclei[start_y:start_y+window_size, start_x:start_x+window_size]
w_cells = cells[start_y:start_y+window_size, start_x:start_x+window_size]
w_cytoplasms = cytoplasms[start_y:start_y+window_size, start_x:start_x+window_size]


### Wrapper Functions

In [11]:
#low level feature extraction - uses cp_measure.bulk.get_core_measurements() to get all basic measurements like intensity, texture, shape, etc.
def get_single_object_features(image, mask, prefix=''):
    """Extract single-object features (Type 1: 1 image + 1 mask).
    
    Args:
        image (np.ndarray): Single channel image
        mask (np.ndarray): Segmentation mask
        prefix (str): Prefix for feature names
        
    Returns:
        dict: Dictionary of extracted features
    """

    results = {} #initializing dict of extracted features
    measurements = get_core_measurements().items()
    # Extract all core measurements
    try:
        # Extract all core measurements except radial distribution
        for name, measure_func in measurements:
            if name != "radial_distribution":  # Skip problematic measurement
                features = measure_func(mask, image)
                # Add prefix to feature names
                if prefix:
                    features = {f'{prefix}_{k}': v for k, v in features.items()}
                results.update(features)
    except Exception as e:
        print(f"Warning: Error in single object measurement: {str(e)}")
        
    return results

#low level feature extraction - extracts all colocalization metrics between two channels within a mask.
def get_colocalization_features(image1, image2, mask, prefix=''):
    """Extract colocalization features (Type 2: 2 images + 1 mask).
    
    Args:
        image1 (np.ndarray): First channel image
        image2 (np.ndarray): Second channel image
        mask (np.ndarray): Segmentation mask
        prefix (str): Prefix for feature names
        
    Returns:
        dict: Dictionary of extracted features
    """

    results = {} #initializing dict of extracted features
    measurements = get_correlation_measurements().items()
    # Extract all correlation measurements
    try:
        # Extract all correlation measurements
        for name, measure_func in measurements:
            features = measure_func(image1, image2, mask) 
            if prefix:
                features = {f'{prefix}_{k}': v for k, v in features.items()}
            results.update(features)
    except Exception as e:
        print(f"Warning: Error in colocalization measurement: {str(e)}")
        
    return results

#low level feature extraction - extracts all neighbor relationship metrics between two masks within a mask.
def get_neighbor_features(mask1, mask2, prefix=''):
    """Extract neighbor relationship features (Type 3: 2 masks).
    
    Args:
        mask1 (np.ndarray): First segmentation mask
        mask2 (np.ndarray): Second segmentation mask
        prefix (str): Prefix for feature names
        
    Returns:
        dict: Dictionary of extracted features
    """
    results = {} #initializing dict of extracted features
    measurements = get_multimask_measurements().items()
    # Extract all correlation measurements
    for name, measure_func in measurements:
        features = measure_func(mask1, mask2,)
        # Add prefix to feature names to ID source
        if prefix:
            features = {f'{prefix}_{k}': v for k, v in features.items()}
        results.update(features)
        
    return results


#INTEGRATION: orchestrates all measurements, matches the existing implementation, catering to Snakemake

# 1 Single-object features for each channel-mask combination
# 2 Colocalization features between all channel pairs
# 3 Neighbor features between different mask types

def extract_phenotype_cp_measure(
    data_phenotype,
    nuclei,
    cells,
    cytoplasms=None,
    channel_names=None,
):
    """Extract comprehensive phenotype features using cp_measure.
    
    Args:
        data_phenotype (np.ndarray): Multi-channel image data (CHANNELS, HEIGHT, WIDTH)
        nuclei (np.ndarray): Nuclear segmentation mask
        cells (np.ndarray): Cell segmentation mask
        cytoplasms (np.ndarray, optional): Cytoplasm segmentation mask
        channel_names (list, optional): List of channel names
        wildcards (dict, optional): Snakemake wildcards
        
    Returns:
        pd.DataFrame: DataFrame containing all extracted features
    """
    # Input validation
    if not isinstance(data_phenotype, np.ndarray) or data_phenotype.ndim != 3:
        raise ValueError("data_phenotype must be 3D array (channels, height, width)")
    for mask, mask_name in [(nuclei, 'nuclei'), (cells, 'cells'), (cytoplasms, 'cytoplasms')]:
        if mask is not None and not isinstance(mask, np.ndarray) or mask.ndim != 2:
            raise ValueError(f"{mask_name} must be a 2D array (height, width)")

    print("Starting feature extraction...")
    # Print input shapes and types
    print("\nInput data properties:")
    print(f"data_phenotype: shape={data_phenotype.shape}, dtype={data_phenotype.dtype}")
    print(f"nuclei: shape={nuclei.shape}, dtype={nuclei.dtype}")
    print(f"cells: shape={cells.shape}, dtype={cells.dtype}")
    print(f"cytoplasms: shape={cytoplasms.shape}, dtype={cytoplasms.dtype}")

    #Naming channels for indexing if channel name list not provided
    if channel_names is None:
        channel_names = [f'ch{i}' for i in range(data_phenotype.shape[0])]
    
    all_features = []
    
    
    try:
        # 1. Single object features
        print("Computing single object features...")
        for channel_idx, channel_name in enumerate(channel_names):
            channel_data = data_phenotype[channel_idx]
            
            # Process each mask type if it contains objects
            for mask, mask_name in [
                (nuclei, 'nucleus'),
                (cells, 'cell'),
                (cytoplasms, 'cytoplasm')
            ]:
                if mask is not None and np.any(mask > 0):
                    features = get_single_object_features(
                        channel_data, mask, f'{mask_name}_{channel_name}'
                    )
                    if features:
                        all_features.append(pd.DataFrame(features))
        
        # 2. Colocalization features
        print("Computing colocalization features...")
        for (ch1_idx, ch1_name), (ch2_idx, ch2_name) in combinations(enumerate(channel_names), 2):
            ch1_data = data_phenotype[ch1_idx]
            ch2_data = data_phenotype[ch2_idx]
            
            for mask, mask_name in [
                (nuclei, 'nucleus'),
                (cells, 'cell'),
                (cytoplasms, 'cytoplasm')
            ]:
                if mask is not None and np.any(mask > 0):
                    features = get_colocalization_features(
                        ch1_data, ch2_data, mask,
                        f'{mask_name}_{ch1_name}_{ch2_name}_coloc'
                    )
                    if features:
                        all_features.append(pd.DataFrame(features))
        
        # 3. Neighbor features
        print("Computing neighbor features...")
        # Process each mask pair if both contain objects
        mask_pairs = [
            (nuclei, cells, 'nucleus_cell'),
            (nuclei, cytoplasms, 'nucleus_cytoplasm'),
            (cells, cytoplasms, 'cell_cytoplasm')
        ]
        
        for mask1, mask2, prefix in mask_pairs:
            if (mask1 is not None and mask2 is not None and 
                np.any(mask1 > 0) and np.any(mask2 > 0)):
                features = get_neighbor_features(mask1, mask2, f'{prefix}_neighbor')
                if features:
                    all_features.append(pd.DataFrame(features))
    
    except Exception as e:
        print(f"Error during feature extraction: {str(e)}")
        return pd.DataFrame()

    # Combine all features
    print("Combining features...")
    if not all_features:
        print("Warning: No features were extracted")
        return pd.DataFrame()
    
    features_df = pd.concat(all_features, axis=1)
    
    return features_df

# # Extract features using cp_measure w/ MOCK DATA
# phenotype_cp = extract_phenotype_cp_measure(
#     data_phenotype=mock_phenotype,
#     nuclei=mock_masks['nuclei'],
#     cells=mock_masks['cells'],
#     cytoplasms=mock_masks['cytoplasm'],
# )

# # Extract features using cp_measure w/ REAL IMG WINDOWS
# phenotype_cp = extract_phenotype_cp_measure(
#     data_phenotype=w_phenotype,
#     nuclei=w_nuclei,
#     cells=w_cells,
#     cytoplasms=w_cytoplasms,
# )

# # Extract features using cp_measure w/ FULL IMAGES
# phenotype_cp = extract_phenotype_cp_measure(
#     data_phenotype=data_phenotype,
#     nuclei=nuclei,
#     cells=cells,
#     cytoplasms=cytoplasms,
# )

# phenotype_cp

### TESTS

In [23]:
# Extract features using cp_measure w/ MOCK DATA
phenotype_cp = extract_phenotype_cp_measure(
    data_phenotype=mock_phenotype,
    nuclei=mock_masks['nuclei'],
    cells=mock_masks['cells'],
    cytoplasms=mock_masks['cytoplasm'],
)

phenotype_cp

Starting feature extraction...

Input data properties:
data_phenotype: shape=(4, 200, 200), dtype=float32
nuclei: shape=(200, 200), dtype=int32
cells: shape=(200, 200), dtype=int32
cytoplasms: shape=(200, 200), dtype=int32
Computing single object features...
Computing colocalization features...
Computing neighbor features...
Combining features...


,nucleus_ch0_RadialDistribution_ZernikeMagnitude_0_0,nucleus_ch0_RadialDistribution_ZernikePhase_0_0,nucleus_ch0_RadialDistribution_ZernikeMagnitude_1_1,nucleus_ch0_RadialDistribution_ZernikePhase_1_1,nucleus_ch0_RadialDistribution_ZernikeMagnitude_2_0,nucleus_ch0_RadialDistribution_ZernikePhase_2_0,nucleus_ch0_RadialDistribution_ZernikeMagnitude_2_2,nucleus_ch0_RadialDistribution_ZernikePhase_2_2,nucleus_ch0_RadialDistribution_ZernikeMagnitude_3_1,nucleus_ch0_RadialDistribution_ZernikePhase_3_1,...,cell_cytoplasm_neighbor_Overlap_TrueNegRate,cell_cytoplasm_neighbor_Overlap_FalseNegRate,cell_cytoplasm_neighbor_Overlap_AdjustedRandIndex,cell_cytoplasm_neighbor_Neighbors_NumberOfNeighbors_Expanded,cell_cytoplasm_neighbor_Neighbors_FirstClosestObjectNumber_Expanded,cell_cytoplasm_neighbor_Neighbors_FirstClosestDistance_Expanded,cell_cytoplasm_neighbor_Neighbors_SecondClosestObjectNumber_Expanded,cell_cytoplasm_neighbor_Neighbors_SecondClosestDistance_Expanded,cell_cytoplasm_neighbor_Neighbors_AngleBetweenNeighbors_Expanded,cell_cytoplasm_neighbor_Neighbors_PercentTouching_Expanded
0,439.845704,1.570796,19.041583,-1.147369,277.574533,-1.570796,23.066841,-1.268659,24.933281,1.462221,...,0.0,0.983558,0.015418,30.0,19,17.374501,23,22.610391,124.151729,99.876238
1,427.099661,1.570796,47.347347,-0.652706,254.877460,-1.570796,5.605238,2.111100,43.953935,2.541630,...,0.0,0.983558,0.015418,45.0,50,8.216441,14,10.465797,32.534463,100.000000
2,425.697432,1.570796,40.449244,2.284852,244.305569,-1.570796,4.466356,2.370637,42.189857,-0.763139,...,0.0,0.983558,0.015418,53.0,5,6.031382,30,11.888046,40.808910,100.000000
3,347.026654,1.570796,31.184507,2.953905,227.550265,-1.570796,3.563208,1.974081,36.266546,-0.153564,...,0.0,0.983558,0.015418,39.0,84,20.093854,86,23.532729,10.187418,99.325084
4,399.273295,1.570796,12.516941,-1.209557,264.979397,-1.570796,3.857968,2.570053,15.422323,1.823971,...,0.0,0.983558,0.015418,54.0,28,13.986868,29,20.118193,19.963532,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,339.747955,1.570796,88.744330,2.117903,88.959272,-1.570796,24.669939,2.948225,45.803143,-2.106730,...,0.0,0.983558,0.015418,27.0,72,8.333904,3,13.092999,50.706663,100.000000
86,130.516140,1.570796,49.861726,-2.673633,33.918052,-1.570796,23.959916,-0.180152,28.212792,-0.305303,...,0.0,0.983558,0.015418,25.0,2,20.830523,58,24.324109,7.697031,100.000000
87,401.640539,1.570796,83.130433,1.712321,172.121557,-1.570796,17.472893,-0.853491,47.485893,-1.871123,...,0.0,0.983558,0.015418,27.0,19,14.619774,65,18.717058,6.228200,100.000000
88,302.297182,1.570796,128.353944,0.907143,102.858178,-1.570796,55.683188,0.189350,99.457509,-2.218435,...,0.0,0.983558,0.015418,30.0,85,17.382675,66,23.204783,178.869735,100.000000


In [12]:
# Extract features using cp_measure w/ REAL IMG WINDOWS
phenotype_cp = extract_phenotype_cp_measure(
    data_phenotype=w_phenotype,
    nuclei=w_nuclei,
    cells=w_cells,
    cytoplasms=w_cytoplasms,
)

phenotype_cp

Starting feature extraction...

Input data properties:
data_phenotype: shape=(4, 200, 200), dtype=uint16
nuclei: shape=(200, 200), dtype=int64
cells: shape=(200, 200), dtype=int64
cytoplasms: shape=(200, 200), dtype=int64
Computing single object features...
Computing colocalization features...
Computing neighbor features...
Error during feature extraction: operands could not be broadcast together with shapes (1,12) (649,649) 


""


In [ ]:
# Extract features using cp_measure w/ FULL IMAGES
phenotype_cp = extract_phenotype_cp_measure(
    data_phenotype=data_phenotype,
    nuclei=nuclei,
    cells=cells,
    cytoplasms=cytoplasms,
)

phenotype_cp